In [2]:
from bs4 import BeautifulSoup
import requests
import chess
import chess.pgn
import requests
import json
import numpy as np
import urllib.parse as up
import re
from tqdm import tqdm
import time

In [3]:
def download_gameknot_game(id):
    url = f"https://gameknot.com/annotate.pl?id={id}"
    results = requests.get(url)
    lines = results.text.split('\n')
    game_movelist = None
    game_notes = None

    game = chess.pgn.Game()
    node = game
    game.headers.clear()

    result_dict = ["", "1-0", "1/2-1/2", "0-1"]

    def fix_move(move):
        if move[-1] == "-":
            return move[:-1]
        return move

    for line in lines:
        if line.startswith('game_movelist'):
            game_str = eval(line[16:-1])
            game_movelist = [game_str[i: i+5] for i in range(0, len(game_str), 5)]
            game_movelist = [fix_move(move) for move in game_movelist]
            # game_movelist = eval(line[16:-1]).split("-")
        elif line.startswith('game_notes'):
            game_notes = list(map(up.unquote, eval(line[13:-1])))
        elif line.startswith('game_result'):
            game.headers["Result"] = result_dict[int(line[-2])]
        elif line.startswith('game_title'):
            game.headers["Event"] = up.unquote(line[25:-3])
        elif line.startswith('game_player_w'):
            game.headers["White"] = up.unquote(line[28:-3])
        elif line.startswith('game_player_b'):
            game.headers["Black"] = up.unquote(line[28:-3])
        elif line.startswith('game_rating_w'):
            game.headers["WhiteElo"] = up.unquote(line[28:-3])
        elif line.startswith('game_rating_b'):
            game.headers["BlackElo"] = up.unquote(line[28:-3])
        elif line.startswith('game_started'):
            game.headers["Date"] = up.unquote(line[27:-3])
        


    node.comment = game_notes[0]


    for move, comment in  zip(game_movelist[:-1], game_notes[1:]):
        node = node.add_variation(chess.Move.from_uci(move))
        node.comment = comment

    return game


# url = 'https://gameknot.com/annotate.pl?id=27351'
game = download_gameknot_game(70528)

print(game)

[Event "Let's play chess"]
[Date "2022.08.17"]
[White "otakumadness"]
[Black "meihua"]
[Result "1-0"]
[BlackElo "994"]
[WhiteElo "1187"]

1. d4 d5 2. e3 e6 3. Nf3 Nf6 4. Nc3 b6 5. Bb5+ c6 6. Bd3 Ng4 7. Ne5 h5 { Very typical opening for me so far. At this point, I'd have expected Nxe5, which I counter with dxe5, but leaving an unprotected pawn out for a possible counterattack, perhaps by Qc7, but my opponent goes for a defensive maneuver instead. } 8. Nxg4 hxg4 9. Qxg4 { With my initiation of the knight exchange, I get a free pawn. } 9... f5 10. Qe2 Qg5 11. g3 e5 12. dxe5 { Getting another free pawn. } 12... Bc5 13. e4 { Getting another free pawn by threatening the queen like this. } 13... Qg6 14. exf5 Qh7 15. f6 Qh3 { At this point my pawn is so close to getting promoted I totally expected gxf6. That does technically allow me to do exf6+, but I'm not sure why my opponent did this instead of doing Qg8 or something else. } 16. Bg6+ Kd8 17. fxg7 { Getting another free pawn. } 17... Rg8 18

In [4]:
def get_game_ids_from_page(page):
    url = f"https://gameknot.com/list_annotated.pl?u=all&c=0&sb=0&rm=0&rn=0&rx=9999&sr=0&p={page}"
    response = requests.get(url)

    ids = re.findall("[?]gm=\d+", response.text)

    ids = set([id[4:] for id in ids])

    return ids

In [5]:
def load_ids_from_file(file_name = "data_scrappers/annotated_game_ids.txt"):
    f = open(file_name)
    return list(map(int, f.readlines()))

all_ids = load_ids_from_file()

In [7]:
failed_ids = []

for i, id in enumerate(tqdm(all_ids)):
    if (i+1)%50 == 0:
        time.sleep(5)
    try:
        game = download_gameknot_game(id)
        if game != None:
            print(game, file=open(f"./data/gameknot/annotated_games_2/{id}.pgn", "w"), end="\n\n")
    except:
        print(id)
        failed_ids.append(id)
    

  1%|          | 112/12751 [00:40<59:29,  3.54it/s]  

23489


  2%|▏         | 221/12751 [01:19<58:17,  3.58it/s]  

10154


  2%|▏         | 250/12751 [01:31<6:11:37,  1.78s/it]